python 3.13.1

Get Naive model on the same tickers used for my sample

In [1]:
import requests
import yfinance as yf
import pandas as pd
import random
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error

/Users/shaneypeterson/Desktop/2024.06.27_-_Data_Science/2024.10.13 - Portfolio Projects/2024.12.04 - Swing Ticker/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
smapes_df_path = '/Users/shaneypeterson/Desktop/2025.02.24 - Data Science Work/combined_model_smape.csv'

In [3]:
smapes_df = pd.read_csv(smapes_df_path)
smapes_df.head()

,ticker,volatility,avg_price,len,smape_standard,smape_tuned,winsorized_pct
0,WPP,Medium,32.136355,9355,0.270578,0.075833,0.364725
1,CVGW,Medium,31.071787,5681,0.162737,0.163886,0.363316
2,DFH,Medium,22.145518,1023,0.305700,0.239105,0.418377
3,RCEL,Medium-High,13.289237,3209,0.915579,0.627327,0.267061
4,C,Medium-Low,79.326248,12132,0.325752,0.241818,0.368035


In [4]:
tickers = smapes_df['ticker'].tolist()
tickers[:5]

['WPP', 'CVGW', 'DFH', 'RCEL', 'C']

In [5]:
def load_data(ticker):
    """
    Downloads historical market data for a given ticker symbol.

    Parameters:
    ticker (str): The ticker symbol of the stock to download data for.

    Returns:
    pd.DataFrame: A DataFrame containing the historical market data for the specified ticker.
    """
    data = yf.download(ticker, period='max') # returns relevant data in df
    data.reset_index(inplace=True) # reset multindex, output is index list of tuples
    cols = list(data.columns) # convert index to list
    cols[0] = ('Date', '') 
    cols = [i[0] for i in cols] # return first element of cols tuples
    data.columns = cols # set as column names
    data['Date'] = pd.to_datetime(data['Date']).dt.date
    data.Date = data.Date.astype('datetime64[ns]')
    return data

In [6]:
df_temp = load_data('AAPL')
df_temp.columns

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [7]:
def get_volatility(data):
    volatility = data.daily_returns.std() * np.sqrt(252)
    if volatility < 0.2:
        category = "Low"
        percentiles=(0.15, 0.85)
    elif volatility < 0.4:
        category = "Medium-Low"
        percentiles=(0.1, 0.9)
    elif volatility < 0.6:
        category = "Medium"
        percentiles=(0.1, 0.9)
    elif volatility < 0.8:
        category = "Medium-High"
        percentiles=(0.05, 0.95)
    else:
        category = "High"
        percentiles=(0.05, 0.95)
    return category, volatility, percentiles

In [8]:
def get_period_params(data, volatility):
    if len(data)/365 < 8:
        period_unit = int(len(data)/4)
        forecast_period = period_unit
        train_period = len(data)
    else:
        period_unit = 365
        forecast_period = period_unit
        train_period = forecast_period * 4 if volatility < 0.6 else forecast_period * 8
    return train_period, period_unit, forecast_period

In [9]:
cv_func = lambda model_name, train_period, period_unit, forecast_period: cross_validation(model_name, 
                                              initial=f'{train_period} days', 
                                              period=f'{period_unit} days', 
                                              horizon=f'{forecast_period} days', 
                                              parallel="processes")

In [10]:
def naive_forecast(history):
    """
    Performs a naive forecast using the last value in the history.

    Parameters:
    history (pd.Series): Time series data to use as history.

    Returns:
    float: The naive forecast (last value of history).
    """
    return history.iloc[-1]

In [11]:
def naive_cross_validation(data, initial, period, horizon):
    """
    Performs cross-validation for a naive forecasting model.

    Parameters:
    data (pd.DataFrame): Time series data with 'Date' and 'Close' columns.
    initial (int): Initial training period length in days.
    period (int): Period between cutoffs in days.
    horizon (int): Forecast horizon in days.

    Returns:
    pd.DataFrame: DataFrame containing actual values, forecasts, and cutoffs.
    """
    initial_date = data['Date'].iloc[0]
    cutoffs = pd.to_datetime([initial_date + pd.Timedelta(days=initial + i * period)
                               for i in range(1 + (len(data) - initial - horizon) // period)])

    periods_list = []
    for cutoff in cutoffs:
        train_df = data[data['Date'] <= cutoff].copy()
        test_df = data[(data['Date'] > cutoff) & (data['Date'] <= cutoff + pd.Timedelta(days=horizon))].copy()

        # if len(test_df) == 0: # Skip if no data in the horizon
        #     continue

        history = train_df['Close']
        forecasts = [naive_forecast(history)] * len(test_df) # Naive forecast for all points in horizon

        test_df['forecast'] = forecasts
        test_df['cutoff'] = cutoff.date() # Store cutoff date

        periods_list.append(test_df)

    df_cv = pd.concat(periods_list).reset_index(drop=True)
    return df_cv

In [12]:
def calculate_smape(cv_results):
    """
    Calculates smape from cross-validation results DataFrame.

    Parameters:
    cv_results (pd.DataFrame): DataFrame from naive_cross_validation.

    Returns:
    float: smape value.
    """
    if cv_results.empty:
        return np.nan  # Return NaN if no cross-validation results

    return np.sqrt(mean_squared_error(cv_results['Close'], cv_results['forecast']))

In [13]:
def get_naive_smape_for_tickers(tickers):
    """
    Calculates naive model smape for a list of stock tickers.

    Parameters:
    tickers (list): List of stock ticker symbols.

    Returns:
    pd.DataFrame: DataFrame with ticker symbols and their naive model smapes.
    """
    naive_smape_results = []
    for ticker in tickers:
        data = load_data(ticker)
        data['daily_returns'] = data.Close.pct_change()
        category, volatility, percentiles = get_volatility(data)
        train_period, period_unit, forecast_period = get_period_params(data, volatility)

        initial_days = train_period
        period_days = period_unit
        horizon_days = forecast_period

        cv_results_naive = naive_cross_validation(
            data[['Date', 'Close']], # Ensure only Date and Close are passed
            initial=period_days,
            period=period_days,
            horizon=horizon_days
        )
        smape_naive = calculate_smape(cv_results_naive)
        naive_smape_results.append({'ticker': ticker, 'naive_smape': smape_naive})
        print(f"Naive smape for {ticker}: {smape_naive:.4f}")

    return pd.DataFrame(naive_smape_results)

In [14]:
naive_smape_df = get_naive_smape_for_tickers(tickers)

[*********************100%***********************]  1 of 1 completed


Naive smape for WPP: 4.8835


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Naive smape for CVGW: 4.1601
Naive smape for DFH: 5.8489


[*********************100%***********************]  1 of 1 completed


...

Naive smape for PBF: 8.8848


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Naive smape for DRN: 2.6732
Naive smape for ULCC: 3.0173


[*********************100%***********************]  1 of 1 completed

Naive smape for AZPN: 9.3325


In [15]:
naive_smape_df.head()

,ticker,naive_smape
0,WPP,4.883490
1,CVGW,4.160058
2,DFH,5.848917
3,RCEL,5.655529
4,C,29.475863


In [19]:
smapes_df = pd.merge(smapes_df, naive_smape_df, on='ticker')

In [20]:
smapes_df.columns

Index(['ticker', 'volatility', 'avg_price', 'len', 'smape_standard',
       'smape_tuned', 'winsorized_pct', 'naive_smape'],
      dtype='object')

In [23]:
# reorder columns

cols = ['ticker', 'volatility', 'avg_price', 'len', 'naive_smape', 'smape_standard', 'smape_tuned', 'winsorized_pct']

smapes_df = smapes_df[cols]
smapes_df.head()

,ticker,volatility,avg_price,len,naive_smape,smape_standard,smape_tuned,winsorized_pct
0,WPP,Medium,32.136355,9355,4.883490,0.270578,0.075833,0.364725
1,CVGW,Medium,31.071787,5681,4.160058,0.162737,0.163886,0.363316
2,DFH,Medium,22.145518,1023,5.848917,0.305700,0.239105,0.418377
3,RCEL,Medium-High,13.289237,3209,5.655529,0.915579,0.627327,0.267061
4,C,Medium-Low,79.326248,12132,29.475863,0.325752,0.241818,0.368035


In [25]:
smape_grid = smapes_df.groupby('volatility').agg({'naive_smape': 'mean', 'smape_standard': 'mean', 'smape_tuned': 'mean'})
smape_grid

,naive_smape,smape_standard,smape_tuned
volatility,,,
High,114156.350363,1.320087,1.063812
Low,2.324579,0.068413,0.046811
Medium,10.642323,0.368252,0.214527
Medium-High,12178.034432,0.552055,0.430741
Medium-Low,9.654945,0.206292,0.130326


In [27]:
# export smape_grid

smape_grid.to_csv('/Users/shaneypeterson/Desktop/2025.02.24 - Data Science Work/smape_grid.csv')

In [28]:
# export smapes_df

smapes_df.to_csv(smapes_df_path, index=False)